<a href="https://colab.research.google.com/github/nakamura196/ndl_ocr/blob/main/Omeka_Classic%E3%81%AEIIIF_Toolkit%E3%82%92%E7%94%A8%E3%81%84%E3%81%9F%E3%82%A2%E3%83%8E%E3%83%86%E3%83%BC%E3%82%B7%E3%83%A7%E3%83%B3%E3%81%AE%E4%B8%80%E6%8B%AC%E7%99%BB%E9%8C%B2%E6%96%B9%E6%B3%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Omeka ClassicのIIIF Toolkitを用いたアノテーションの一括登録方法

以下の記事も参考にしてください。

https://zenn.dev/nakamura196/articles/042d4c8e960171

## 事前準備0

Omeka ClassicのAPIの有効化、およびAPIキーの発行を行なってください。上記記事も参考にしてください。

## 設定1

In [ ]:
# Omeka ClassicのURL
omeka_url = "https://omeka.aws.ldas.jp" # 例

# APIキー
api_key = "XXXXXXXXXXXXXXXXXXXXXX"

# Omeka Classicにログインするためのユーザ情報

# ユーザ名
username = "demo" # 例
# パスワード
password = "ZZZZZZZZZZZ" # 例

## 事前準備1

アノテーションの付与対象となるIIIFマニフェストファイル（v2）をOmeka Classicに登録しておきます。登録方法については、以下の記事などを参考にしてください。

https://zenn.dev/nakamura196/books/2a0aa162dcd0eb/viewer/99c318


今回は、以下の「金沢貞顕書状（金沢文庫所蔵）」を一部改変して利用します。

https://www.kanzaki.com/works/2016/pub/image-annotator?u=https%3A%2F%2Fgist.githubusercontent.com%2Fnakamura196%2Fc83367eb3a415bad3195b143125821eb%2Fraw%2Fe2a5b45f5fed0d84374837075ac62702c5318c08%2Fmanifest.json

（公開元）国宝　金沢文庫文書データベース

https://kanazawabunko-db.pen-kanagawa.ed.jp/item/376

## 事前準備2

登録対象とするアノテーションの一覧を含むIIIFマニフェストファイル（v3）を用意します。今回は、以下に示すアノテーションデータを含むファイルを使用します。

https://www.kanzaki.com/works/2016/pub/image-annotator?u=https://gist.githubusercontent.com/nakamura196/66ca2f52efa0673355f32d3f1883efae/raw/a28bb2f8b534cf14e6499031c2f223369c8fc75d/manifest3.json

注意点として、canvasのidが、「事前準備1」で用意したIIIFマニフェストファイル（v2）と対応するようにしてください。

（準備の方法については、別途記述予定。）

## 設定2

事前準備に基づき、以下を設定します。

In [ ]:
# 事前準備1の結果、Omeka Classicに登録されたコレクションのID
collection_id = 1 # 例

# 　事前準備2で用意した、登録対象とするアノテーションの一覧を含むIIIFマニフェストファイル
manifest_v3_url = "https://gist.githubusercontent.com/nakamura196/66ca2f52efa0673355f32d3f1883efae/raw/a28bb2f8b534cf14e6499031c2f223369c8fc75d/manifest3.json" # 例

## 実行

### omekaのitemのidと、canvas_idの対応関係の取得

以下のAPIを利用します。

{omeka_url}/api/items?collection={collection_id}&key={api_key}

In [ ]:
import pprint

base = "{}/api/items?collection={}&key={}".format(omeka_url, collection_id, api_key)

page = 1

canvas_id_map = {}

while 1:

  url = "{}&page={}".format(base, page)

  df = requests.get(url).json()

  if len(df) == 0:
    break

  page += 1

  for item in df:
    item_id = item["id"]

    element_texts = item["element_texts"]

    for element_text in element_texts:
      if element_text["element"]["name"] == "Original @id":
        canvas_id = element_text["text"]
        canvas_id_map[canvas_id] = item_id

pprint.pprint(canvas_id_map)

### アノテーション一覧の取得

In [ ]:
df = requests.get(manifest_v3_url).json()

items = df["items"]

annotationMap = {}

for item in items:
  canvas_id = item["id"]
  annotations = item["annotations"][0]["items"]
  values = []
  for a in annotations:
    values.append({
        "label": "item",
        "xywh": a["target"].split("=")[1]
        })
  annotationMap[canvas_id] = values

pprint.pprint(annotationMap)

## 登録

### ログイン

In [ ]:
from bs4 import BeautifulSoup
import requests
import time

payload = {
    'username': username,
    'password': password
}

# authenticity_tokenの取得
s = requests.Session()
s.get(omeka_url)

# ログイン
r = s.post('{}/admin/users/login'.format(omeka_url), data=payload)

print(r.text)

### アノテーションの追加

以下を参考にしています。

https://github.com/utlib/IiifItems/wiki/The-Mirador-Omeka-Annotator-Endpoint

tagやpublic（公開・非公開設定）などは適宜変更してください。

In [ ]:
import json

tag = "字形"
chars = "item"
public = True
featured = False


for canvas_id in annotationMap:
  item_id = canvas_id_map[canvas_id]
  url = "{}/admin/iiif-items/annotator/items/{}".format(omeka_url, item_id)
  for a in annotationMap[canvas_id]:
    xywh = a["xywh"]
    x, y, w, h = xywh.split(",")

    item = {
      "@context": "http://iiif.io/api/presentation/2/context.json",
      "@type": "oa:Annotation",
      "motivation": [
        "oa:tagging",
        "oa:commenting"
      ],
      "resource": [
        {
          "@type": "oa:Tag",
          "chars": tag
        },
        {
          "@type": "dctypes:Text",
          "format": "text/html",
          "chars": "<p>{}</p>".format(chars)
        }
      ],
      "_iiifitems_access": {
        "public": public,
        "featured": featured
      },
      "on": [
        {
          "@type": "oa:SpecificResource",
          "full": canvas_id,
          "selector": {
            "@type": "oa:Choice",
            "default": {
              "@type": "oa:FragmentSelector",
              "value": "xywh={}".format(xywh)
            },
            "item": {
              "@type": "oa:SvgSelector",
              "value": '''<svg xmlns='http://www.w3.org/2000/svg'><path xmlns=\"http://www.w3.org/2000/svg\" d=\"M{},{}h{}v{}h-{}v-{}z\" id=\"rectangle_{}\" fill-opacity=\"0\" fill=\"#00bfff\" fill-rule=\"nonzero\" stroke=\"#00bfff\" stroke-width=\"1\" stroke-linecap=\"butt\" stroke-linejoin=\"miter\" stroke-miterlimit=\"10\" stroke-dasharray=\"\" stroke-dashoffset=\"0\" font-family=\"none\" font-weight=\"none\" font-size=\"none\" text-anchor=\"none\" style=\"mix-blend-mode: normal\"/></svg>'''.format(x, y, w, h, w, h, "abc")
            }
          },
          "within": {
            "@id": "{}/oa/items/{}/manifest.json".format(base, item_id),
            "@type": "sc:Manifest"
          }
        }
      ]
    }

    data = json.dumps(item)
    res = s.post(url, data=data)